# Sample Tail Simulations

In [1]:
import sys
import os
import glob

# Add user site-packages to the system path
user_site_packages = os.path.expanduser("~/.local/lib/python*/site-packages")
sys.path.extend(glob.glob(user_site_packages)) # Use glob to handle potential version differences

import ergodic_insurance as ei

ei.__version__

'0.3.0'

In [2]:
from tqdm.notebook import tqdm  # progress bar
from itertools import product
import os
from joblib import Parallel, delayed
import shutil # Import shutil for file operations

from run_tail_simulation_colab import run_tail_simulation

# Integrate joblib with tqdm for per-task progress
class TqdmJoblib(tqdm):
    def __enter__(self):
        from joblib import parallel
        self._original = parallel.BatchCompletionCallBack
        outer = self
        class TqdmBatchCompletionCallback(parallel.BatchCompletionCallBack):
            def __call__(self, *args, **kwargs):
                outer.update(n=self.batch_size)
                return super().__call__(*args, **kwargs)
        parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
        return super().__enter__()
    def __exit__(self, exc_type, exc, tb):
        from joblib import parallel
        parallel.BatchCompletionCallBack = self._original
        return super().__exit__(exc_type, exc, tb)

NUM_SIMULATIONS = 1_000
SIM_YEARS = 25
PRICING_SIMULATIONS = 5_000  # For premium estimation

initial_assets_list = [10_000_000]
atr_list = [1.0]
ebitabl_list = [0.1]
deductible_list = [100_000]
loss_ratio_list = [0.3, 0.4, 0.5, 0.6, 0.7]
policy_limit_list = [50_000_000, 100_000_000, 200_000_000, 500_000_000]
extreme_threshold_percentile = [0.001, 0.0005, 0.0001]
extreme_sev_shape = [1.0, 1.5, 2.0, 2.5]
extreme_sev_scale_percentile = [0.5, 1.0, 1.5, 2.0]  # Scale at the 1/10th of the threshold percentile of catastrophic severity

# Test overrides
loss_ratio_list = [0.7]
policy_limit_list = [50_000_000]
extreme_threshold_percentile = [0.0005]
extreme_sev_shape = [1.5]
extreme_sev_scale_percentile = [1.5]  # Scale at the 1/10th of the threshold percentile of catastrophic severity

# Use all but one CPU core
_cpu = os.cpu_count() or 1
n_jobs = max(_cpu, 1)

print(f'Available cores: {n_jobs}')

# Define the destination directory in your Google Drive
# Make sure 'My Drive/results' exists in your Google Drive
gdrive_results_dir = 'output'
os.makedirs(gdrive_results_dir, exist_ok=True) # Create the directory if it doesn't exist

prod_with_extreme_thresholds = product(
                                    initial_assets_list,
                                    atr_list,
                                    ebitabl_list,
                                    deductible_list,
                                    loss_ratio_list,
                                    policy_limit_list,
                                    extreme_threshold_percentile,
                                    extreme_sev_shape,
                                    extreme_sev_scale_percentile
                                )

prod_without_extreme_thresholds = product(
                                    initial_assets_list,
                                    atr_list,
                                    ebitabl_list,
                                    deductible_list,
                                    loss_ratio_list,
                                    policy_limit_list,
                                    [None],
                                    [0.0],
                                    [1.0]
                                )

prod_combined = list(prod_with_extreme_thresholds) + list(prod_without_extreme_thresholds)

total_combos = len(prod_combined)

# Parallel execution with progress bar
with TqdmJoblib(total=total_combos, desc="Parameter sets", leave=True):
    Parallel(n_jobs=n_jobs, prefer="processes", batch_size=1)(
        delayed(run_tail_simulation)(
            index, ia, atr, ebitabl, ded, lr, policy_limit,
            extreme_threshold, extreme_sev_shape, extreme_sev_scale,
            NUM_SIMULATIONS, SIM_YEARS, PRICING_SIMULATIONS,
            filepath=gdrive_results_dir
        )
        for index, (ia, atr, ebitabl, ded, lr, policy_limit,
                    extreme_threshold, extreme_sev_shape, extreme_sev_scale) in enumerate(prod_combined)
    )

Available cores: 16


Parameter sets:   0%|          | 0/2 [00:00<?, ?it/s]